In [856]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data=260
df=pd.read_csv("Data.csv", header=True)
data_raw=np.array(df, dtype=float)
val_data=data_raw[train_data:300,:]
data=data_raw[0:train_data, :]
#np.random.shuffle(data)
#scaler = MinMaxScaler()
scaler = StandardScaler()
data = scaler.fit_transform(data_raw)

In [ ]:
train_data=260
df=pd.read_csv("Data.csv", header=True)
data_imp=np.array(df, dtype=float)
scaler=StandardScaler()
data_raw=scaler.fit_transform(data_imp)
val_data=data_raw[train_data:300,:]
data=data_raw[0:train_data, :]
data

array([[ 1.51927551,  0.25133476, -0.41744385,  1.06862162, -0.7967402 ],
       [-0.63522413, -1.41272158, -0.28728364, -1.24330016,  1.80848396],
       [ 0.24153596,  1.20157556,  0.41766566, -1.24330016,  1.12239001],
       ...,
       [ 0.28401798,  0.18367842,  1.0640143 ,  1.06862162, -0.844476  ],
       [ 0.96524668,  1.66901907, -1.67735295, -1.24330016,  1.69624072],
       [-0.89930591,  0.7551262 , -0.72449683, -1.24330016,  1.35092074]])

In [859]:

s=20  #sampling size
seed_size=20 #size of seed data
q=5 #query size
n=int((train_data-seed_size)/s) # number of iteration needed to go through all samples

In [860]:
#Defining seed data
seed_data=data[0:seed_size,:]
new_data=pd.DataFrame(seed_data)
k=seed_data.shape[0]

In [861]:
#GP model for random sampling
def gp_eval(X, y):
    # Instantiate a Gaussian Process model
    #kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    kernel = 1.0 * Matern(length_scale=0.1, nu=1.5)
    gp_eval = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

    # Fit to data using Maximum Likelihood Estimation of the parameters
    gp_eval.fit(X, y)
    r2_score=gp.score(X,y)
    print("GP model fit score is=",r2_score)
    #RMSE Error evaluation
    y_val_pred=gp_eval.predict(val_data[:, 0:4])
    tt=np.sqrt(np.mean((val_data[:, 4]-y_val_pred)**2))
    print("Random sampling RMSE value is =",tt)
    
    return tt

In [862]:
    X=data[:, 0:4]
    y=data[:,4]
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    # Fit to data using Maximum Likelihood Estimation of the parameters
    gp.fit(X, y)
    r2_score=gp.score(X,y)
    print("GP model fit score is=",r2_score)
    y_val_pred=gp.predict(val_data[:, 0:4])
    rmse_cal=np.sqrt(np.mean((val_data[:, 4]-y_val_pred)**2))
    mse_cal=np.mean((val_data[:, 4]-y_val_pred)**2)
    print("Final rmse value is =",rmse_cal)

GP model fit score is= 1.0
Final rmse value is = 0.16342810026157117


In [863]:
mse=[]
mse_random=[]
rmse=[]
rmse_random= []
for i in range(n):
    X=new_data.iloc[:, 0:4]
    y=new_data.iloc[:,4]
    # Instantiate a Gaussian Process model
    #kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

    # Fit to data using Maximum Likelihood Estimation of the parameters
    gp.fit(X, y)
    r2_score=gp.score(X,y)
    print("GP model fit score is=",r2_score)
    #Evaluate current model against the validation set
    #RMSE Error evaluation
    y_val_pred=gp.predict(val_data[:, 0:4])
    rmse_cal=np.sqrt(np.mean((val_data[:, 4]-y_val_pred)**2))
    mse_cal=np.mean((val_data[:, 4]-y_val_pred)**2)
    print("Final rmse value is =",rmse_cal)
    rmse.append(rmse_cal)
    mse.append(mse_cal)
    #pool-based sampling from unlabeled pool
    XX=data[k+i*s:k+(i+1)*s, 0:4]   #taking the next few samples to predict variance
    y_pred,sigma =gp.predict(XX, return_std=True)  #predict variance
    sigma
    #Finding maximum variance of the new samples,Rank them and select top 5
    var_pred=pd.DataFrame(sigma, columns=["sigma"])
    new_input=pd.DataFrame(data[k+i*s:k+(i+1)*s,0:5])
    md=pd.concat([new_input, var_pred], axis=1)
    sorted_data=md.sort_values(by=['sigma'], ascending=False)
    #select top 5
    c_data=sorted_data.iloc[0:q,:]
    #concat the ne
    df_1=pd.DataFrame(new_data)
    new_data=pd.concat([df_1, c_data], axis=0)
    vv=new_data.shape[0]
    print(vv)

GP model fit score is= 1.0
Final rmse value is = 0.39454448438709555
25
GP model fit score is= 1.0
Final rmse value is = 0.33002484839563856
30
GP model fit score is= 1.0
Final rmse value is = 0.30219905673152014
35
GP model fit score is= 1.0
Final rmse value is = 0.2752617053416001
40
GP model fit score is= 1.0
Final rmse value is = 0.3069681871029265
45
GP model fit score is= 1.0
Final rmse value is = 0.29450843950902017
50
GP model fit score is= 1.0
Final rmse value is = 0.24770711679436042
55
GP model fit score is= 1.0
Final rmse value is = 0.233632464075003
60
GP model fit score is= 1.0
Final rmse value is = 0.23193318486481293
65
GP model fit score is= 1.0
Final rmse value is = 0.23789476973577148
70
GP model fit score is= 1.0
Final rmse value is = 0.23485669330717696
75
GP model fit score is= 1.0
Final rmse value is = 0.23666486639142006
80


In [864]:
#random learning from the original pool
rmse_rand_orig= []
mse_rand_orig= []
for i in range(n):
        df_seed=pd.DataFrame(data[0:seed_size,:])
        if i==0:
            rand_rmse=gp_eval(df_seed.iloc[0:seed_size, 0:4], df_seed.iloc[0:seed_size, 4])
        else:
            df_unlabel=pd.DataFrame(data[seed_size:train_data,:])
            #df_unlabel=df.iloc[20:400,0:2]
            df_add=df_unlabel.sample(n=q*i)
            data_random=pd.concat([df_seed, df_add], axis=0)
            rand_rmse=gp_eval(data_random.iloc[:, 0:4], data_random.iloc[:, 4])
            ll=data_random.shape[0]
            print(ll)
            print(i)

        rmse_rand_orig.append(rand_rmse)
print(rmse_rand_orig)

GP model fit score is= 1.0
Random sampling RMSE value is = 0.39454448467635217
GP model fit score is= 0.9906084662361287
Random sampling RMSE value is = 0.35763049963243293
25
1
GP model fit score is= 0.98568265319165
Random sampling RMSE value is = 0.24178514378012073
30
2
GP model fit score is= 0.9922954456559969
Random sampling RMSE value is = 0.28573160143170717
35
3
GP model fit score is= 0.9605168917930804
Random sampling RMSE value is = 0.48635161792190007
40
4
GP model fit score is= 0.9725198207687386
Random sampling RMSE value is = 0.22928654855815161
45
5
GP model fit score is= 0.9506442136782994
Random sampling RMSE value is = 0.28577155285616446
50
6
GP model fit score is= 0.8083514859514467
Random sampling RMSE value is = 0.3685039077009337
55
7
GP model fit score is= 0.982106333327674
Random sampling RMSE value is = 0.24323265520407883
60
8
GP model fit score is= 0.9477740570542649
Random sampling RMSE value is = 0.3236026994432029
65
9
GP model fit score is= 0.9684118639

In [867]:
#save model
import pickle

# save
#with open('model_20_5_30.pkl','wb') as f:
    #pickle.dump(gp,f)
filename = f'model_{seed_size}_{q}_{s}.pkl'  # Using f-string to insert variables into the filename
with open(filename, 'wb') as f:
    pickle.dump(gp, f)
file_path = f'scaler_{seed_size}_{q}_{s}.pkl'

# Save the scaler using pickle.dump
with open(file_path, 'wb') as file:
    pickle.dump(scaler, file)

print(f'Scaler saved to file: {file_path}')
# load
#with open('model.pkl', 'rb') as f:
#    gp2 = pickle.load(f)

#gp2.predict(val_data[:, 0:3])

Scaler saved to file: scaler_20_5_20.pkl


In [868]:
# Load the model from the saved file
with open(filename, 'rb') as f:
    loaded_model = pickle.load(f)
